In [2]:
import pandas as pd

from datetime import timedelta, date
from quantbullet.utils.data import generate_fake_bond_trades
from quantbullet.model_selection.rolling_window import RollingWindowManager, RollingWindowBacktester
from quantbullet.model import Model, Feature, FeatureSpec, FeatureRole, DataType

In [3]:
%load_ext autoreload
%autoreload 2

## Generate Synthetic Data

In [6]:
START_DATE = date( 2025, 1, 1 )
END_DATE = date( 2025, 4, 30 )

In [7]:
df = generate_fake_bond_trades( start_date=START_DATE, end_date=END_DATE )

In [8]:
df.head()

,date,ticker,rating,feature_A,feature_B,feature_C,feature_D,expiry,yield
0,2025-01-01,TICK046 AAA,AAA,1.037280,0.750856,-1.445716,-0.434421,2030-09-20,7.699485
1,2025-01-01,TICK022 AAA,AAA,-1.349441,-0.449716,-0.543036,0.858139,2029-01-08,2.125824
2,2025-01-01,TICK035 AAA,AAA,1.477746,-0.121104,-2.318184,-0.211448,2025-07-16,5.899495
3,2025-01-01,TICK049 AAA,AAA,1.271709,-1.039925,-0.298122,-1.388109,2034-05-13,3.868152
4,2025-01-01,TICK047 AAA,AAA,1.324646,0.622864,-0.196974,-0.319635,2029-11-23,6.484171


## Define Model and Features

In [9]:
AverageYieldModelFeatureSpec = FeatureSpec(
    [
        Feature( "rating", DataType.CATEGORICAL, FeatureRole.MODEL_INPUT),
        Feature( "yield", DataType.FLOAT, FeatureRole.TARGET)
    ]
)

In [10]:
class AverageYieldModel( Model ):
    def __init__( self, feature_spec: FeatureSpec ):
        super().__init__( feature_spec )

    def fit( self, X, y=None ):
        self._rating_map = X.groupby( "rating" ).agg( { "yield": "mean" } ).reset_index()
        self._rating_map = self._rating_map.set_index( "rating" ).to_dict()

    def predict( self, X ):
        return X[ "rating" ].map( self._rating_map[ "yield" ] ).values

In [11]:
rwm = RollingWindowManager.from_flat( df, window_size=30 )
model = AverageYieldModel( feature_spec=AverageYieldModelFeatureSpec )
backtester = RollingWindowBacktester( rolling_window_manager=rwm, model=model )

In [12]:
backtester.run( use_multithreading=False )

In [13]:
backtester.get_logger_df().head()

,date,ticker,rating,feature_A,feature_B,feature_C,feature_D,expiry,yield,loggerPred,loggerRunDate,loggerResid
0,2025-01-31,TICK024 BBB,BBB,0.083753,0.511852,0.470484,-0.407524,2026-05-10,6.078283,5.138238,2025-01-30,0.940045
1,2025-01-31,TICK048 BBB,BBB,-1.020937,0.012302,-0.551115,-1.993220,2032-10-26,4.210123,5.138238,2025-01-30,-0.928116
2,2025-01-31,TICK022 BB,BB,-1.938274,-0.096296,-0.488395,-0.114678,2026-05-24,2.867997,4.968971,2025-01-30,-2.100974
3,2025-01-31,TICK044 BB,BB,0.308720,1.293961,-1.430654,0.436478,2025-10-22,6.451625,4.968971,2025-01-30,1.482654
4,2025-01-31,TICK005 BB,BB,-0.522593,0.093999,-0.874177,1.252807,2028-05-21,4.807759,4.968971,2025-01-30,-0.161211
